from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme("solarizedd")

In [177]:
import numpy as np
import pygame as p

In [178]:
# Constants:
WIDTH, HEIGHT = 400, 400
ROWS, COLS, = 8, 8
BOX_SIZE = WIDTH//COLS
RED, GREEN, BLUE = (255,0,0), (0,255,0), (0,0,255)
WHITE, BLACK, SILVER = (255, 255, 255), (0,0,0), (192,192,192)

image_dimensions = 17
CROWN = p.transform.scale(p.image.load("crown.png"), (image_dimensions,image_dimensions))

In [179]:
class Piece:

  PADDING = 14
  BORDER = 2

  def __init__(self, row, column, colour):
    self.row = row
    self.column = column
    self.colour = colour
    if self.colour == WHITE:
      self.direction = "+"
    else:
      self.direction = "-"
    self.king = False
    self.x = 0
    self.y = 0
    self.calculate_position()

  def set_king(self):
    self.king = True

  def calculate_position(self):
    self.x = BOX_SIZE * self.column + BOX_SIZE // 2
    self.y = BOX_SIZE * self.row + BOX_SIZE // 2

  def draw(self, win):
    radius = BOX_SIZE//2 - self.PADDING
    p.draw.circle(win, SILVER, (self.x,self.y), radius + self.BORDER)
    p.draw.circle(win, self.colour, (self.x,self.y), radius)
    if self.king:
      new_x = self.x - CROWN.get_width()//2
      new_y = self.y - CROWN.get_height()//2
      win.blit(CROWN, (new_x, new_y))

  def move_piece(self, row, column):
    self.row = row
    self.column = column
    self.calculate_position()

  def __repr__(self):
    return self.direction

In [180]:
class GameBoard(object):


  def __init__(self):
    self.board = np.zeros(shape=(8,8)).astype("int").tolist()
    self.player_count = 0
    self.selected_piece = None
    self.red_remaining = self.white_remaining = 12
    self.red_king_count = self.white_king_count = 0


  def setup_board(self):
    # Start player 2 at "top" of board.
    colour = 2
    # Counter to switch to using player 1 identifier.
    counter = 0 
    ln = len(self.board)
    for row_index in range(ln):
      if self.is_even(row_index) and row_index != 4:
        self.board[row_index] = [colour,0,colour,0,colour,0,colour,0]
      elif not self.is_even(row_index) and row_index != 3:
        self.board[row_index] = [0,colour,0,colour,0,colour,0,colour]
      counter += 1
      if counter == 4:
        colour = 1
    # Messy Setup.
    for row_index in range(ln):
      for col_index in range(ln):
        if self.board[row_index][col_index] != 0:
          colour = self.board[row_index][col_index]
          if colour == 1: 
            colour = RED
          else: 
            colour = WHITE
          self.board[row_index][col_index] = Piece(row_index, col_index, colour) 


  def draw_board(self, win):
    win.fill(RED)
    for row in range(ROWS):
      for col in range(row % 2, COLS, 2):
        p.draw.rect(win, BLACK, (row*BOX_SIZE, col*BOX_SIZE, BOX_SIZE, BOX_SIZE))


  def draw_all(self, win):
    # Fill Board with pieces.
    self.setup_board()
    # Draw board.
    self.draw_board(win)
    # Draw pieces on board.
    for row in range(ROWS):
      for col in range(COLS):
        piece = self.board[row][col]
        if piece != 0:
          piece.draw(win)


  def get_valid_moves(self, Piece, direction):
    # End function if non player piece.
    try:
      # Get row, column (indicies) from tuple object piece.
      row, column = Piece.row, Piece.column
    except AttributeError:
      print("No valid moves for an empty space!")
      return
    # Get player 1 or 2.
    player = self.board[row][column]
    # Potential next moves list.
    next_move_list = []
    # Next row - dependent on player.
    next_row = None
    if direction == "+":
      next_row = row + 1
      next_next_row = row + 2
    elif direction == "-":
      next_row = row - 1
      next_next_row = row - 2
    # List to hold columns on either side.
    left_right = [column-1,column+1]
    # Loop through left right options.
    for next_col in left_right:
      if next_col in range(8):
        # Check state of potential next square.
        # Case: Empty square.
        if self.whats_in_the_box(next_row, next_col) == 0:
          next_move_list.append((next_row, next_col))
        # Case: Opponent-Occupied square.
        # If next space is opponent and next next space is clear.
        elif self.whats_in_the_box(next_row, next_col) != player: # This could be refined to check if == opponent. 
          # Assign next next column indicies.
          next_next_col = None
          if next_col == column - 1:
            next_next_col = column - 2
          else:
            next_next_col = column + 2
          if self.whats_in_the_box(next_next_row,next_next_col) == 0:
            next_move_list.append((next_next_row, next_next_col))
    return next_move_list  


  def move_piece(self, Piece, new_row, new_col):
    row, col = Piece.row, Piece.column
    self.remove_piece(row,col)
    Piece.move_piece(new_row, new_col)
    self.board[new_row][new_col] = Piece
    # King update.
    if (new_row == 0 or new_row == ROWS - 1) and Piece.king != True:
      Piece.set_king()
      if Piece.colour == RED:
        self.red_king_count += 1
      else:
        self.white_king_count += 1

  def remove_piece(self,row, col):
    self.board[row][col] = 0

  def whats_in_the_box(self, row, col):
    return self.board[row][col]

  def is_even(self,num):
    return (num % 2) == 0 

  def print_board(self):
    print()
    for row in self.board:
      print(row)
    print()

In [181]:
# Test Cell:
gb = GameBoard()
gb.setup_board()
gb.print_board()

piece = gb.whats_in_the_box(2,0)
print(piece.king)

potential_moves = gb.get_valid_moves(piece, piece.direction)

# gb.move_piece(piece, potential_moves[0])
gb.move_piece(piece, 7,1)

gb.print_board()

print(piece.king)


[+, 0, +, 0, +, 0, +, 0]
[0, +, 0, +, 0, +, 0, +]
[+, 0, +, 0, +, 0, +, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, -, 0, -, 0, -, 0, -]
[-, 0, -, 0, -, 0, -, 0]
[0, -, 0, -, 0, -, 0, -]

False

[+, 0, +, 0, +, 0, +, 0]
[0, +, 0, +, 0, +, 0, +]
[0, 0, +, 0, +, 0, +, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, -, 0, -, 0, -, 0, -]
[-, 0, -, 0, -, 0, -, 0]
[0, +, 0, -, 0, -, 0, -]

True


In [182]:
p.display.init()
SCREENSIZE = (WIDTH, HEIGHT)
WIN = p.display.set_mode(SCREENSIZE)
p.display.set_caption("CHECKERZZZ")
FPS = 30

def main():
    run = True
    clock = p.time.Clock()
    gb = GameBoard()
    gb.setup_board()

    piece = gb.whats_in_the_box(0,0)
    gb.move_piece(piece, 7,7)

    while run:
        # Maintain constant frames/second.
        clock.tick(FPS)
        # Look for events during run.
        for event in p.event.get():
            if event.type == p.QUIT:
                run = False

            if event.type == p.MOUSEBUTTONDOWN:
                pass

            gb.draw_all(WIN)
            p.display.update()
            p.display.flip()
        
    p.display.quit()

main()

[2, 0, 2, 0, 2, 0, 2, 0] 

[0, 2, 0, 2, 0, 2, 0, 2] 

[2, 0, 2, 0, 2, 0, 2, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 1, 0, 1, 0, 1, 0, 1] 

[1, 0, 1, 0, 1, 0, 1, 0] 

[0, 1, 0, 1, 0, 1, 0, 1] 